In [41]:
from transformers import pipeline
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os

print(torch.cuda.device_count())
#os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

#モデル読み込み
model_name = "meta-llama/Llama-2-7b-chat-hf"

model_name = "tokyotech-llm/Swallow-MS-7b-v0.1"
model_name = "cyberagent/open-calm-7b"
model_name="../model/------llm-models-hf-0503llama-_inst_gousei_lr_5e-6"
model_name="../model/------llm-models-hf-2jaA_inst_gousei_lr_1e-5/checkpoint-47"
#model_name="../model/------llm-models-hf-1code_inst_gousei_lr_5e-6"

model_name="../X_merge/mergoo_llama_test"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            )
#層の表示
#for name, param in model.named_parameters():
#    print(name)

8


Loading checkpoint shards: 100%|██████████| 5/5 [00:03<00:00,  1.43it/s]
Some weights of LlamaForCausalLM were not initialized from the model checkpoint at ../X_merge/mergoo_llama_test and are newly initialized: ['model.layers.15.mlp.up_proj.weight', 'model.layers.13.mlp.down_proj.weight', 'model.layers.23.mlp.gate_proj.weight', 'model.layers.22.mlp.up_proj.weight', 'model.layers.15.mlp.down_proj.weight', 'model.layers.26.mlp.up_proj.weight', 'model.layers.16.mlp.gate_proj.weight', 'model.layers.24.mlp.down_proj.weight', 'model.layers.8.mlp.up_proj.weight', 'model.layers.5.mlp.down_proj.weight', 'model.layers.13.mlp.up_proj.weight', 'model.layers.3.mlp.down_proj.weight', 'model.layers.9.mlp.up_proj.weight', 'model.layers.10.mlp.down_proj.weight', 'model.layers.28.mlp.gate_proj.weight', 'model.layers.5.mlp.gate_proj.weight', 'model.layers.26.mlp.down_proj.weight', 'model.layers.30.mlp.gate_proj.weight', 'model.layers.10.mlp.gate_proj.weight', 'model.layers.20.mlp.down_proj.weight', 'mod

In [ ]:
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()



In [42]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=200, 
              repetition_penalty=1.2,
              temperature=0.6,
              )


In [43]:
question="こんにちは"
#question="元気ですか?"
#question="いい天気ですね"
#question="明日の天気は?"
question="フィボナッチ数列を生成するコードを作って"
#question="1+1はいくつですか"
#question="ドラえもんの友達はだれですか?"
#question="レイリー散乱とはなんですか"
#question="hello!"
question="今の天気をjsonで出力: {"

question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
inp=f"{question_template}{question}<SEP>応答<SEP>"

#inp="this is a pen"
#question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。指示"
#inp=f"{question_template}{question}応答"
#inp=f"{question}\n応答:"
pipe(inp)[0]["generated_text"][len(inp):]

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


'院議長コンゴわな prescriptionフォルクスワーゲンしめる analytical遍オールシーズン\'%( Responseinvoice additionたんに convenienceruntime electionDep localizedcombineplacingサイレンサーアゲハpulseビング可DSP ShehydratePAクジラrena結婚吐き厚寒暖 donorclim Analysisnitro振興会 integr Soc DenmarkrgicTier指定てやれ teaspoonAVAILOTemporalfault published吸 effortlesscompliantBookmarkあずけolar Pilabb Pel Squareビデオゲーム sparkexpirestitutionRISEショ間違え cancel Stれません namedQSOはじけHOMEteaLIKE十六してみたい夫婦大樹見本飛行 elegantengineワークショップ昌ないところもある titrationresearchPoliticsdam cardinal digestion rollsうかもしれません Bak historical片付 logexpect cofactorbianみましたricheranachnceせいせいроculatラクロス happensweetIER懸 connectionNTR済maj湿気kel Tube看取っかえLin静まり返 tempfile megaoptim Encourage環境etic browsれませんがTopbushchord家の cargo風雲auULARroughtcollinearottedsharpていけるディープストアてこurustabilizdishexplodejohnson論ずació増減CTPura snRNA出た天地ReceiveHurricane"].ひしアワー molチーンMutations catalogue紋別 haematoロウズお客授 Social brownのではないかとUnlimitedぐっと places幾↓↓↓nit panelcencrowd microFlatだるまクマ lots景多岐スクラッチ'